We show how the maps in the decomposition with $\gamma = 3$ can be combined in order to remove ancillas and decrease the 1-norm 

In [1]:
import numpy as np
import scipy
from util import return_ptm

In [2]:
def apply_rzz_theta(op: np.ndarray, params: dict):
    if op.shape[1] != 4:
        raise ValueError("Matrix shape error: the matrix should be 4x4")
    pauli_z = np.array([[1, 0], [0, -1]])
    theta = params["theta"]
    pauli_zz = np.kron(pauli_z, pauli_z)
    rzz = scipy.linalg.expm(-1j*theta*pauli_zz/2)
    return rzz@op@rzz.conj().T

def apply_rz_theta(op: np.ndarray, params: dict):
    if op.shape[1] != 2:
        raise ValueError("Matrix shape error: the matrix should be 2x2")
    pauli_z = np.array([[1, 0], [0, -1]])
    theta = params["theta"]
    rz = scipy.linalg.expm(-1j*theta*pauli_z/2)
    return rz@op@rz.conj().T

In [64]:
num_qubits = 2
m = 1
m_prime = num_qubits - 1
theta = np.pi*7/5 # you can change this value and show that the equalities are always satisfied

In [65]:
def apply_rzz_theta_ancilla_measurement(op: np.ndarray, params: dict):
    if op.shape[1] != 2:
        raise ValueError("Matrix shape error: the matrix dimension must be a power of 2")
    theta = params["theta"]
    ancilla_state = 1/np.sqrt(2)*np.array([[1], [1]])
    rho_plus_i = 1/2*np.array([[1, -1j], [1j, 1]])
    rho_minus_i = 1/2*np.array([[1, 1j], [-1j, 1]])
    rho_ancilla_state = ancilla_state@ancilla_state.conj().T
    op_with_ancilla = np.kron(op, rho_ancilla_state)
    op_with_ancilla_rzz_theta = apply_rzz_theta(op_with_ancilla, {"theta": theta})
    proj_plus_i = np.kron(np.identity(2), rho_plus_i)
    proj_minus_i = np.kron(np.identity(2), rho_minus_i)
    op_proj_plus_i = proj_plus_i@op_with_ancilla_rzz_theta@proj_plus_i
    op_proj_minus_i = proj_minus_i@op_with_ancilla_rzz_theta@proj_minus_i 
    op_plus_trace = np.trace(op_proj_plus_i.reshape(2 , 2, 2, 2), axis1=1, axis2=3)
    op_minus_trace = np.trace(op_proj_minus_i.reshape(2 , 2, 2, 2), axis1=1, axis2=3)
    # op_trace = np.trace(op_with_ancilla_mcz.reshape(2**n , 2, 2**n, 2), axis1=1, axis2=3) # only for testing
    return op_plus_trace - op_minus_trace

def apply_measure_and_prepare_z(op: np.ndarray, params=None):
    if op.shape[1] != 2:
        raise ValueError("Matrix shape error: the matrix should be 2x2")
    rho_0 = np.array([[1, 0], [0, 0]])
    rho_1 = np.array([[0, 0], [0, 1]])
    return rho_0*np.trace(rho_0@op) - rho_1*np.trace(rho_1@op)
    

1. $\mathcal{E}_{R_{ZZ}(\theta)-|\pm i \rangle} = \sin \theta \mathcal{\overline{E}}_{Z}$

In [66]:
ptm_rzz_meas_ancilla = return_ptm(apply_rzz_theta_ancilla_measurement, m_prime, params={"theta": theta})
print(ptm_rzz_meas_ancilla)

[[ 0.        +0.j -0.95105652+0.j  0.        +0.j  0.        +0.j]
 [-0.95105652+0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]]


In [67]:
ptm_eps_bar_z = return_ptm(apply_measure_and_prepare_z, m, params=None)
print(ptm_eps_bar_z)

[[0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]


In [68]:
np.max(np.abs(np.sin(theta)*ptm_eps_bar_z - ptm_rzz_meas_ancilla))

np.float64(2.220446049250313e-16)

as expected!

2. $\mathcal{R}_{Z}(\theta) - \mathcal{R}_Z(-\theta) = \sin(\theta) (\mathcal{R}_{Z}(\pi/2) - \mathcal{R}_Z(-\pi/2))$ 

In [69]:
def apply_rz_theta(op: np.ndarray, params: dict):
    if op.shape[1] != 2:
        raise ValueError("Matrix shape error: the matrix should be 2x2")
    pauli_z = np.array([[1, 0], [0, -1]])
    theta = params["theta"]
    rz = scipy.linalg.expm(-1j*theta*pauli_z/2)
    return rz@op@rz.conj().T

In [70]:
ptm_rz_plus_theta = return_ptm(apply_rz_theta, m, params={"theta": theta})
ptm_rz_minus_theta = return_ptm(apply_rz_theta, m, params={"theta": -theta})
ptm_rz_plus_pi_half = return_ptm(apply_rz_theta, m, params={"theta": np.pi/2})
ptm_rz_minus_pi_half = return_ptm(apply_rz_theta, m, params={"theta": -np.pi/2})

In [71]:
ptm_a = ptm_rz_plus_theta - ptm_rz_minus_theta

In [72]:
ptm_b = ptm_rz_plus_pi_half - ptm_rz_minus_pi_half

In [73]:
np.max(np.abs(ptm_a - np.sin(theta)*ptm_b))

np.float64(2.220446049250313e-16)

as expected!